<a href="https://colab.research.google.com/github/mallhartotey2903-png/Decibal-Duel-mallhar-250041026/blob/main/Task%202%20%3A%20Audio_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================
# 🎧 DECIBEL DUEL
# ================

!pip install -q kaggle librosa tensorflow pandas scikit-learn tqdm

import os, glob, librosa, numpy as np, pandas as pd, tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras import layers, models
from google.colab import files

# --------------------------------------------------------------
# STEP 1️⃣: Upload Kaggle API key
# --------------------------------------------------------------
print("📂 Upload your kaggle.json file (Kaggle → Account → Create API Token)")
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("✅ Kaggle API configured!")

# --------------------------------------------------------------
# STEP 2️⃣: Download & Extract Dataset
# --------------------------------------------------------------
!kaggle competitions download -c the-frequency-quest -p /content/
!unzip -q /content/the-frequency-quest.zip -d /content/dataset
print("✅ Dataset downloaded and extracted!")

# --------------------------------------------------------------
# STEP 3️⃣: Auto-detect nested train/train & test/test
# --------------------------------------------------------------
train_candidates = glob.glob("/content/dataset/**/[Tt]rain", recursive=True)
test_candidates = glob.glob("/content/dataset/**/[Tt]est", recursive=True)

TRAIN_DIR = sorted(train_candidates, key=lambda x: x.count('/'))[-1]
TEST_DIR  = sorted(test_candidates, key=lambda x: x.count('/'))[-1]

print("✅ Using Train folder:", TRAIN_DIR)
print("✅ Using Test folder:", TEST_DIR)

# --------------------------------------------------------------
# STEP 4️⃣: Audio → Mel-spectrogram
# --------------------------------------------------------------
SR = 22050
DURATION = 3
N_MELS = 128
IMG_SIZE = (128, 128)

def extract_mel(file):
    try:
        y, sr = librosa.load(file, sr=SR, duration=DURATION)
        mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
        mel_db = librosa.power_to_db(mel, ref=np.max)
        mel_db = (mel_db + 80) / 80.0   # normalize 0–1 range
        mel_db = np.resize(mel_db, IMG_SIZE)
        return mel_db
    except Exception:
        return None

X, y = [], []

print("\n🎵 Extracting features from training data...")
for label in os.listdir(TRAIN_DIR):
    class_dir = os.path.join(TRAIN_DIR, label)
    if not os.path.isdir(class_dir): continue

    files = [f for f in os.listdir(class_dir) if f.lower().endswith(('.wav', '.mp3', '.ogg'))]
    print(f"📁 {label}: {len(files)} files")

    for file in tqdm(files, desc=f"Processing {label}"):
        mel = extract_mel(os.path.join(class_dir, file))
        if mel is not None:
            X.append(mel)
            y.append(label)

if len(X) == 0:
    raise ValueError("❌ No audio files were successfully loaded. Please check dataset path or file format!")

X = np.array(X)[..., np.newaxis]
le = LabelEncoder()
y_encoded = tf.keras.utils.to_categorical(le.fit_transform(y))

print(f"✅ Loaded {len(X)} samples across {len(le.classes_)} classes: {list(le.classes_)}")

# --------------------------------------------------------------
# STEP 5️⃣: Train/Validation Split
# --------------------------------------------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# --------------------------------------------------------------
# STEP 6️⃣: CNN Model
# --------------------------------------------------------------
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),   # slightly higher dropout
    layers.Dense(y_encoded.shape[1], activation='softmax')
])

# --------------------------------------------------------------
# STEP 7️⃣: Training Strategy
# --------------------------------------------------------------

# ✅ Warm-up: Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=24,    # smaller batch for smoother convergence
    shuffle=True,
    verbose=1
)

from tensorflow.keras.optimizers import SGD
optimizer = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

# Callbacks for better generalization
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
    )
]

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Fine-tuning model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=60,
    batch_size=24,
    shuffle=True,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Highest Validation Accuracy:", round(max(history.history['val_accuracy'])*100, 2), "%")

# --------------------------------------------------------------
# STEP 8️⃣: Predict Test Set
# --------------------------------------------------------------
print("\n🎧 Processing test data...")
X_test, test_files = [], sorted(os.listdir(TEST_DIR))

for file in tqdm(test_files):
    mel = extract_mel(os.path.join(TEST_DIR, file))
    if mel is None:
        mel = np.zeros(IMG_SIZE)
    X_test.append(mel)

X_test = np.array(X_test)[..., np.newaxis]
pred = model.predict(X_test)
pred_labels = le.inverse_transform(np.argmax(pred, axis=1))

submission = pd.DataFrame({"ID": test_files, "Class": pred_labels})
submission.to_csv("submission.csv", index=False)
print("\n💾 submission.csv created successfully!")

from google.colab import files
files.download("submission.csv")

print("\n🏁 DONE! Upload 'submission.csv' to Kaggle.")


📂 Upload your kaggle.json file (Kaggle → Account → Create API Token)


Saving kaggle.json to kaggle.json
✅ Kaggle API configured!
100% 2.68G/2.68G [00:36<00:00, 20.5MB/s]
100% 2.68G/2.68G [00:36<00:00, 80.0MB/s]
✅ Dataset downloaded and extracted!
✅ Using Train folder: /content/dataset/train/train
✅ Using Test folder: /content/dataset/test/test

🎵 Extracting features from training data...
📁 street_music: 700 files


Processing street_music: 100%|██████████| 700/700 [00:26<00:00, 26.29it/s]


📁 siren: 650 files


Processing siren: 100%|██████████| 650/650 [00:09<00:00, 65.66it/s]


...

100%|██████████| 740/740 [00:14<00:00, 50.14it/s]


24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step

💾 submission.csv created successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🏁 DONE! Upload 'submission.csv' to Kaggle.


In [ ]:
# Save processed data after extraction
np.save("/content/X_features.npy", X)
np.save("/content/y_labels.npy", y_encoded)

print("✅ Saved feature data for next run!")

# Load pre-extracted feature data
X = np.load("/content/X_features.npy")
y_encoded = np.load("/content/y_labels.npy")

print("✅ Loaded saved features, ready to train!")



✅ Saved feature data for next run!
✅ Loaded saved features, ready to train!


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),   # slightly higher dropout
    layers.Dense(y_encoded.shape[1], activation='softmax')
])

# --------------------------------------------------------------
# STEP 7️⃣: Training Strategy (Minor Tweaks)
# --------------------------------------------------------------

# ✅ Warm-up: Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=24,    # smaller batch for smoother convergence
    shuffle=True,
    verbose=1
)

# ✅ Fine-tune with SGD (momentum)
from tensorflow.keras.optimizers import SGD
optimizer = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

# Callbacks for better generalization
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
    )
]

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Fine-tuning model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=60,
    batch_size=24,
    shuffle=True,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Highest Validation Accuracy:", round(max(history.history['val_accuracy'])*100, 2), "%")

# --------------------------------------------------------------
# STEP 8️⃣: Predict Test Set
# --------------------------------------------------------------
print("\n🎧 Processing test data...")
X_test, test_files = [], sorted(os.listdir(TEST_DIR))

for file in tqdm(test_files):
    mel = extract_mel(os.path.join(TEST_DIR, file))
    if mel is None:
        mel = np.zeros(IMG_SIZE)
    X_test.append(mel)

X_test = np.array(X_test)[..., np.newaxis]
pred = model.predict(X_test)
pred_labels = le.inverse_transform(np.argmax(pred, axis=1))

submission = pd.DataFrame({"ID": test_files, "Class": pred_labels})
submission.to_csv("submission.csv", index=False)
print("\n💾 submission.csv created successfully!")

from google.colab import files
files.download("submission.csv")

print("\n🏁 DONE! Upload 'submission.csv' to Kaggle.")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.4806 - loss: 11.5232 - val_accuracy: 0.2029 - val_loss: 36.7268
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.5980 - loss: 2.6534 - val_accuracy: 0.3623 - val_loss: 6.9707
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6677 - loss: 1.5299 - val_accuracy: 0.2841 - val_loss: 12.6150
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7021 - loss: 1.1871 - val_accuracy: 0.3449 - val_loss: 16.2078
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7433 - loss: 1.0434 - val_accuracy: 0.5971 - val_loss: 1.6872
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7527 - loss: 0.7629 - val_accuracy: 0.7290 - val_loss: 1.0089
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.7888 - loss: 0.6013 - val_accuracy: 0.7000 - val_loss: 1.3019
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8410 - loss: 0.5510 - val

100%|██████████| 740/740 [00:12<00:00, 60.10it/s]


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step

💾 submission.csv created successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🏁 DONE! Upload 'submission.csv' to Kaggle.
